In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.1 MB 15.2 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 
     |████████████████████████████████| 59 kB 8.9 MB/s 
     |████████████████████████████████| 895 kB 70.4 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
     |████████████████████████████████| 1.2 MB 12.3 MB/s 


In [2]:
# Mount Google drive to upload datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The path to the data on my drive
D = '/content/drive/My Drive/W266_Project_Data/pmi_data'

In [4]:
import pandas as pd
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [5]:
# Load the gold-standard target translation datasets
hi_target = pd.read_csv(D+"/predicted_text/MBart/dev.hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ml_target = pd.read_csv(D+"/predicted_text/MBart/dev.ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ta_target = pd.read_csv(D+"/predicted_text/MBart/dev.ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])

In [6]:
hi_target.head()

,sentence_1
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [7]:
# Load the Bart translated files into their respective language datasets
hi_bart = pd.read_csv(D+"/predicted_text/MBart/new_dev_bart_translated_hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])
ml_bart = pd.read_csv(D+"/predicted_text/MBart/new_dev_bart_translated_ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])
ta_bart = pd.read_csv(D+"/predicted_text/MBart/new_dev_bart_translated_ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])

In [8]:
hi_bart.head()

,sentence_2
0,Prime Minister said Babasaheb Ambedkar has a k...
1,आज बीजापुर में इस समारोह को आयोजित करने के महत...
2,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों..."


In [9]:
# Create Hindi Bart test set
hi_bart_test = pd.concat([hi_target['sentence_1'], hi_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
hi_bart_test.insert(0, 'language_task', 'translate English to Hindi')
hi_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों..."


In [10]:
# Create Malayalam Bart test set
ml_bart_test = pd.concat([ml_target['sentence_1'], ml_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ml_bart_test.insert(0, 'language_task', 'translate English to Malayalam')

In [11]:
# View first few rows of Malayalam Bart test set
ml_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...
1,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...
2,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...
3,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...
4,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക..."


In [12]:
# Load the first few rows of the Tamil Bart test set
ta_bart_test = pd.concat([ta_target['sentence_1'], ta_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ta_bart_test.insert(0, 'language_task', 'translate English to Tamil')

In [13]:
# View the first few rows of the Tamil translation
ta_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Tamil,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,"கோடிக்கணக்கான மக்களின் இதயங்களிலும், மனங்களிலு..."
1,translate English to Tamil,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,இந்த நிகழ்ச்சியை இன்று பிஜப்பூர் நகரில் நடத்து...
2,translate English to Tamil,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022-ம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ந...
3,translate English to Tamil,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுகளுடன் பல்வ...
4,translate English to Tamil,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த வகையில் ஜன் தன் கணக்குகள், ஏழைகளுக்கு காப..."


In [15]:
class SiameseNet(torch.nn.Module):
    def __init__(self):
      super(SiameseNet, self).__init__()

      # Set up a fully-connected layer to apply to each incoming sentence vector
      # Use a dense layer (nn.Linear) to reduce the dimensionality of the input embeddings
      # use a relu activation function
      # Randomly drop out
      # Set 1 as the dimensionality of the final output since we want to predict the logits corresponding to a binary label
      # Initialize the feed forward layer
      self.fc = torch.nn.Sequential(
            torch.nn.Linear(768, 128),
            torch.nn.LayerNorm(128),
            torch.nn.ReLU(inplace=True))
      
      self.relu = torch.nn.ReLU(inplace=True)
      self.batch2 = torch.nn.LayerNorm(384)
      self.ffn = torch.nn.Linear(384, 128)
      self.batch3 = torch.nn.LayerNorm(128)
      self.layer_out = torch.nn.Linear(128,1)

      # Add drop out
      self.drop = torch.nn.Dropout(0.2)

    # left and right represent columns in our data
    def forward(self, left, right):

      # Get the sentence embeddings 
      u = left
      v = right

      # Reduce the dimensionality
      u = self.fc(u)
      v = self.fc(v)

      # Get the absolute value of the difference of the vectors
      uv = torch.sub(u, v)
      uv_abs = torch.abs(uv)

      # concatenate u, v, and the absolute value of the element-wise difference between the embeddings
      x = torch.cat([u, v, uv_abs], dim=-1)

      # Normalize the concatenated batch
      x = self.batch2(x)

      # Use the relu activation function
      x = self.relu(x)

      # Pass the concatenated layers through a dense layer to reduce the dimensionality from 2304 to 128
      x = self.ffn(x)
      # Normalize the concatenated layers
      x = self.batch3(x)
      # Pass through relu activation
      x = self.relu(x)
      # Implement drop out to avoid overfitting
      x = self.drop(x)
      # Pass the output through a final dense layer
      output = self.layer_out(x)
      # Reshape the output to have the same dimension as the target labels
      output = output[:,-1,:]

      # Concatenate the 
      return output

In [16]:
# Set up the device to run on GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [17]:
# Create an instance of the tokenizer and model using Indic Bert
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', keep_accents=True)
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.weight', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def sentence_embeddings(device, tokenizer, model, sentences):
  """Create contextualized sentence embeddings
  
  Parameters:
  tokenizer: transformers tokenizer for creating word tokens
  model:
  sentences: sentences to generate sentence embeddings for
  """

  # Tokenize sentences and return PyTorch tensors
  encoded_input = tokenizer(sentences, padding=True, truncation=True, 
                            max_length=128, return_tensors='pt')
  encoded_input = encoded_input.to(device)
  
  # Pass the tokenized input to the model 
  with torch.no_grad():
    model.to(device)
    model_output = model(**encoded_input) 
  
  # Perform mean pooling to get total sentence embeddings
  sentence_embeddings = mean_pooling(model_output, 
                                     encoded_input['attention_mask'])
  
  return sentence_embeddings

In [19]:
# Load the Siamese Neural Net from the pre-trained weights
net = SiameseNet().to(device)
# Load the weights and biases from the last round of training
net.load_state_dict(torch.load("/content/drive/My Drive/W266_Project_Data/models/siamese_model/siamese_models_used/three_dense_w_punjabi_model_layernorm.pt"))

<All keys matched successfully>

In [20]:
# Create a dataset class, similarly to the one used for training the Siamese Neural Network dataset - the difference is that it won't create labels
class TranslationEvalDataset(Dataset): 
  def __init__(self,training_df, device, tokenizer, model):
    self.training_df=training_df
    self.sent1 = self.training_df['sentence_1'].tolist()
    self.sent2 = self.training_df['sentence_2'].tolist()
    self.language_task = self.training_df['language_task'].tolist()
    self.device = device
    self.tokenizer = tokenizer
    self.model = model

  def __getitem__(self, index):
    sent1 = self.sent1[index]
    sent2 = self.sent2[index]
    task = self.language_task[index]

    # Get the sentence embeddings for sentences 1 and 2 for each row
    sent1_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent1)
    sent2_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent2)
    return sent1_embeddings, sent2_embeddings, task

  def __len__(self):
    return len(self.training_df)

In [21]:
# Concatenate all of the bart test sets into one test set
final_test = pd.concat([hi_bart_test, ml_bart_test, ta_bart_test]).reset_index()
final_test.drop(columns={'index'}, inplace=True)

In [22]:
final_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों..."


In [23]:
final_test[1000:2000]

,language_task,sentence_1,sentence_2
1000,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...
1001,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...
1002,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...
1003,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...
1004,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക..."
...,...,...,...
1995,translate English to Malayalam,മ്യാന്‍മാറിലെ യൂണിയന്‍ തിരഞ്ഞെടുപ്പ് കമ്മിഷന്‍ 7.,vi. മിയാമാൻ യൂണിയൻ എLECTION COMMISSION;
1996,translate English to Malayalam,സ്വാതന്ത്ര്യാനന്തരം പല നേട്ടങ്ങളും ഉണ്ടാക്കാന്...,சுதந்திரത്തിന് ശേഷം ഒരുപാട് നേട്ടങ്ങള് ഉണ്ടായി...
1997,translate English to Malayalam,"ജന്‍ ഭാഗീദാരി, വിവരം പങ്കിടല്‍, വിഭവങ്ങള്‍ പരമ...","അദ്ദേഹം അവരോട് സംസാരിച്ചു, നല്ല ഭരണത്തിന് റെ ച..."
1998,translate English to Malayalam,"ഗ്രാമ സ്വരാജ് അഭിയാന്‍, ആയുഷ്മാന്‍ ഭാരത് തുടങ്...","ഗർമൻ സ്വraj Abhiyan, ആയുഷ്മൻ பாரத் എന്നിവ അண்ம..."


In [24]:
final_test.tail()

,language_task,sentence_1,sentence_2
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,vi. மியான்மர் மத்திய தேர்தல் ஆணையம்;
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,சுதந்திரத்திற்குப் பிறகு பல சாதனைகள் ஏற்பட்டுள...
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...",ஜன் பாகிதாரி உள்ளிட்ட நல்ல நிர்வாகத்தின் சில அ...
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","கிராம சுயாட்சி இயக்கம், ஆயுஷ்மான் பாரத் போன்ற ..."
2999,translate English to Tamil,இந்தியாவின் 130 லட்சம் மக்கள் சார்பில் உங்கள் ...,இந்தியாவின் மூன்று பில்லியன் மக்களின் சார்பில்...


In [25]:
# Create an instance of the test set with all test translations
bart_test = TranslationEvalDataset(final_test, device, tokenizer, model)

In [26]:
# Create an instance of the test data loader
bart_test_loader = DataLoader(bart_test, batch_size=16)

In [27]:
# Track the language_tasks and sentences  being evaluated
y_language_task = final_test.language_task.tolist()
y_sent1 = final_test.sentence_1.tolist()
y_sent2 = final_test.sentence_2.tolist()
probability_of_paraphrase = []

with torch.no_grad():

  # Place the model into evaluation mode
    net.eval()

    # Loop over the test set
    for i, data in enumerate(bart_test_loader, 0):

      # Extract the test set components
      sent1_test, sent2_test, task_test = data

      # Send the test data to GPU
      sent1_test, sent2_test = sent1_test.to(device), sent2_test.to(device)

      # Get the test set predictions
      test_pred = net(sent1_test, sent2_test)
            
      # Get the predicted test labels
      y_test_pred = torch.sigmoid(test_pred).float()

      # Add the test labels to their respective list
      probability_of_paraphrase.extend(y_test_pred)

In [28]:
# Create a dataframe of the tasks, sentences, and probabilities of the second sentence (the translated one) being a paraphrase
y_probs = [x.item() for x in probability_of_paraphrase]
bart_performance = pd.DataFrame(list(zip(y_language_task, y_sent1, y_sent2, y_probs)),columns =['language_task', 'target_sentence', 'bart_translation', 'probability_of_paraphrase'])

In [29]:
# Check the first few rows
bart_performance.head()

,language_task,target_sentence,bart_translation,probability_of_paraphrase
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,0.066522
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,0.981713
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,0.990610
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.987714
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",0.975804


In [30]:
# Sort the dataframe in order of lowest predicted probability and see where BART returns the lowest scores
# The lowest quality translations, as measured by paraphrase score, are those of Malayalam and Tamil
bart_performance.sort_values("probability_of_paraphrase")

,language_task,target_sentence,bart_translation,probability_of_paraphrase
1531,translate English to Malayalam,അത്തരം ഇഴുകിച്ചേരലുകള്‍ നമുക്കിടയിലുള്ള ബന്ധം ...,ഇത്തരം പരസ്പര ബന്ധങ്ങള് നമ്മുടെ ബന്ധങ്ങളെ സജ്ജ...,0.008471
545,translate English to Hindi,इसमें Ease of doing business के साथ ही Ease of...,व्यवसाय करने में आसानता और जीवन में आसानता इस ...,0.009105
1045,translate English to Malayalam,സാമ്പത്തിക പരിവര്‍ത്തനത്തിനും വികസനത്തിനും ഇന്...,"ഇന്ത്യയുടെ പുരോഗതിയിൽ നിന്നും, സമ്പന്നതയിൽ നിന...",0.009592
1720,translate English to Malayalam,"റോഡുകള്‍, റെയില്‍വേ, ഒപ്റ്റിക്കല്‍ ഫൈബര്‍ ശൃംഖ...",He also mentioned the increase in pace in vari...,0.010443
2496,translate English to Tamil,ரஷ்யாவின் தீவிரவாத்திற்கு எதிரான பார்வையே எங்க...,G20 க்குள் நமது ஒத்துழைப்பை மேம்படுத்தவும் நாங...,0.010659
...,...,...,...,...
777,translate English to Hindi,उन्होंने कहा कि पेरिस समझौते को अंतिम रूप दिए ...,उन्होंने कहा कि पेरिस समझौते को अंतिम रूप देने...,0.990867
118,translate English to Hindi,यह एक ऐतिहासिक उपलब्धि है।,यह एक ऐतिहासिक उपलब्धि है।,0.990898
792,translate English to Hindi,मैं 18वीं भारत-रूस वार्षिक शिखर वार्ता के लिए ...,मैं 18वीं भारत-रूस वार्षिक शिखर सम्मेलन के लिए...,0.991114
821,translate English to Hindi,प्रधानमंत्री ने पुल के निर्माण में शामिल सभी ल...,प्रधानमंत्री ने पुल के निर्माण में शामिल सभी ल...,0.991165


In [31]:
# Check the number of rows with a predicted probability <0.5 (meaning that they would be assigned a label of 0 - not a paraphrase)
len(bart_performance[bart_performance.probability_of_paraphrase < 0.5])

1469

In [32]:
# Check the number of translations of each language where the translation is not counted as a paraphrase
hi_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Hindi')])
ml_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Malayalam')])
ta_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Tamil')])
print(hi_errors)
print(ml_errors)
print(ta_errors)

100
802
567


In [33]:
# The comparatively high error rate for Malayalam and Tamil reflect their comparatively lower Sacre Bleu scores
hi_error_rate = (hi_errors/1000)
ml_error_rate = (ml_errors/1000)
ta_error_rate = (ta_errors/1000)

In [34]:
# Hindi performs well
print("{0:.2%}".format(hi_error_rate))

10.00%


In [35]:
# Malayalam error rate
print("{0:.2%}".format(ml_error_rate))

80.20%


In [36]:
# The Tamil error rate is high, but not as high as Malayalam
print("{0:.2%}".format(ta_error_rate))

56.70%


In [37]:
# Get the sentence embeddings for a pair of "dissimilar sentences"
sent1_embed = sentence_embeddings(device, tokenizer, model, bart_performance['target_sentence'][2433])
sent2_embed = sentence_embeddings(device, tokenizer, model, bart_performance['bart_translation'][2433])

In [38]:
# Find the cosine similarity score between these pair of "dissimilar sentences"
cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

print(cosine_similarity_value)

tensor(0.6589, device='cuda:0')


In [39]:
# Try for another pair of sentences, this time, for those with a high similarity score
sent3_embed = sentence_embeddings(device, tokenizer, model, bart_performance['target_sentence'][1])
sent3_embed = sentence_embeddings(device, tokenizer, model, bart_performance['bart_translation'][1])

In [40]:
# The sentences are the same
F.cosine_similarity(sent3_embed.squeeze(0), sent3_embed.squeeze(0), dim=0)

tensor(1., device='cuda:0')

In [41]:
# Get the sentence embeddings and cosine similarity scores between each pair of sentences in the dataset
# This is to check if low scoring sentences have exact similarity scores or if they are of low quality
def check_cosine_similarity(df, device, tokenizer, model):

  # Create an empty list to check cosine similarity scores
  cosine_scores = []

  for index, rows in df.iterrows():
    # Get the sentence embeddings of each pair of target sentences
    sent1_embed = sentence_embeddings(device, tokenizer, model, rows['target_sentence'])
    sent2_embed = sentence_embeddings(device, tokenizer, model, rows['bart_translation'])

    # Reshape the embeddings to be of a single dimension 
    # Get the cosine similarity between the two pairs of sentence embeddings
    cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

    # Add the cosine similarity score to the list of scores
    cosine_scores.append(cosine_similarity_value.item())

  return cosine_scores

In [42]:
translation_scores = check_cosine_similarity(bart_performance, device, tokenizer, model)

In [43]:
# Add the translation similarity scores to the indictrans performance dataframe
bart_performance['cosine_similarity_scores'] = translation_scores

In [44]:
# Sentneces with low cosine similarity also have low probability of paraphrase
bart_performance.sort_values('cosine_similarity_scores')

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
1434,translate English to Malayalam,സര്‍വ്വേ നടത്തപ്പെട്ടവരില്‍ ശരാശരി 70 ശതമാനം പ...,survey survey survey survey survey survey surv...,0.024460,0.198134
1218,translate English to Malayalam,സ്വച്ഛ് ഭാരത് പദ്ധതിക്കായി പ്രധാനമന്ത്രി നടത്ത...,delegation members also praised the Prime Mini...,0.013488,0.349155
1449,translate English to Malayalam,ഇതുപോലെ ഇത് ലോകത്തിലെ എല്ലാ രാജ്യങ്ങളിലും ആഘോഷ...,Guru Nanak Dev Ji's 550th Prakash Parv similar...,0.010852,0.354075
2992,translate English to Tamil,இறந்தவர்கள் குடும்பத்தினருக்கு எனது இரங்கலை தெ...,எனது thoughts are with the bereaved families.,0.014369,0.382268
1501,translate English to Malayalam,ചടങ്ങിനെത്തിയവരെ അഭിസംബോധന ചെയ്യവേ കഴിഞ്ഞ നാലു...,"shrouding the gathering, the Prime Minister re...",0.010829,0.414973
...,...,...,...,...,...
2090,translate English to Tamil,பாதுகாப்பு மற்றும் இணைய பாதுகாப்பு,பாதுகாப்பு மற்றும் இணைய பாதுகாப்பு,0.987277,1.000000
118,translate English to Hindi,यह एक ऐतिहासिक उपलब्धि है।,यह एक ऐतिहासिक उपलब्धि है।,0.990898,1.000000
394,translate English to Hindi,हमारे प्रमुख राष्ट्रीय प्रतीकों ने बौद्ध धर्म ...,हमारे प्रमुख राष्ट्रीय प्रतीकों ने बौद्ध धर्म ...,0.990118,1.000000
2687,translate English to Tamil,"எனதருமை நாட்டுமக்களே, வணக்கம்.","எனதருமை நாட்டுமக்களே, வணக்கம்.",0.986803,1.000000


In [45]:
# High similarity scores have a high probability of paraphrase
bart_performance.sort_values('cosine_similarity_scores', ascending=False)

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
2090,translate English to Tamil,பாதுகாப்பு மற்றும் இணைய பாதுகாப்பு,பாதுகாப்பு மற்றும் இணைய பாதுகாப்பு,0.987277,1.000000
90,translate English to Hindi,रक्षा और साइबर सुरक्षा,रक्षा और साइबर सुरक्षा,0.988091,1.000000
2687,translate English to Tamil,"எனதருமை நாட்டுமக்களே, வணக்கம்.","எனதருமை நாட்டுமக்களே, வணக்கம்.",0.986803,1.000000
118,translate English to Hindi,यह एक ऐतिहासिक उपलब्धि है।,यह एक ऐतिहासिक उपलब्धि है।,0.990898,1.000000
2272,translate English to Tamil,மூன்று தேசிய நெடுஞ்சாலை திட்டங்களுக்கு அவர் அட...,மூன்று தேசிய நெடுஞ்சாலை திட்டங்களுக்கு அவர் அட...,0.988605,1.000000
...,...,...,...,...,...
1501,translate English to Malayalam,ചടങ്ങിനെത്തിയവരെ അഭിസംബോധന ചെയ്യവേ കഴിഞ്ഞ നാലു...,"shrouding the gathering, the Prime Minister re...",0.010829,0.414973
2992,translate English to Tamil,இறந்தவர்கள் குடும்பத்தினருக்கு எனது இரங்கலை தெ...,எனது thoughts are with the bereaved families.,0.014369,0.382268
1449,translate English to Malayalam,ഇതുപോലെ ഇത് ലോകത്തിലെ എല്ലാ രാജ്യങ്ങളിലും ആഘോഷ...,Guru Nanak Dev Ji's 550th Prakash Parv similar...,0.010852,0.354075
1218,translate English to Malayalam,സ്വച്ഛ് ഭാരത് പദ്ധതിക്കായി പ്രധാനമന്ത്രി നടത്ത...,delegation members also praised the Prime Mini...,0.013488,0.349155


In [46]:
# Find the range in cosine similarity scores for sentences not considered paraphrases of each other
np = bart_performance[bart_performance.probability_of_paraphrase < 0.5].reset_index()
np.drop(columns={'index'}, inplace=True)

In [47]:
# Find the min and max of the cosine similarity scores
print(min(np.cosine_similarity_scores))
# There are some sentences labeled as not paraphrases of each other that have a high degree of cosine similarity
print(max(np.cosine_similarity_scores))

0.1981336921453476
0.9816931486129761


In [48]:
# Do the same for sentences considered paraphrases of each other
p = bart_performance[bart_performance.probability_of_paraphrase >= 0.5].reset_index()
print(min(p.cosine_similarity_scores))
print(max(p.cosine_similarity_scores))

0.5870420932769775
1.0


In [49]:
# Look at the pair of sentences with the highest degree of cosine similarity
np[np['cosine_similarity_scores'] > 0.5]

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,0.066522,0.770237
1,translate English to Hindi,सेना ने हमेशा देश को प्राथमिकता दी है।,हमारी सेना हमेशा राष्ट्र को सबसे पहले रखती है।,0.494800,0.845936
2,translate English to Hindi,प्रधानमंत्री ने कहा कि रेल कोच मरम्‍मत कारखाना...,Prime Minister said that Rail Coach Refurbishi...,0.099778,0.773225
3,translate English to Hindi,पृष्ठ भूमिः,पृष्ठभूमिः,0.340932,0.638530
4,translate English to Hindi,प्रधानमंत्री ने कहा कि भारत सरकार जलमार्गों की...,Prime Minister said that the Government of Ind...,0.074620,0.787075
...,...,...,...,...,...
1463,translate English to Tamil,இது ஆப்கானிஸ்தானின் பன்முக கலாச்சாரத்தின் மீது...,அவர்கள் ஆப்கானிஸ்தானின் பல்வகை பண்பாட்டு அமைப்...,0.114356,0.925376
1465,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,சுதந்திரத்திற்குப் பிறகு பல சாதனைகள் ஏற்பட்டுள...,0.043672,0.878720
1466,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...",ஜன் பாகிதாரி உள்ளிட்ட நல்ல நிர்வாகத்தின் சில அ...,0.083110,0.932954
1467,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","கிராம சுயாட்சி இயக்கம், ஆயுஷ்மான் பாரத் போன்ற ...",0.344651,0.942588


In [50]:
np.sort_values('probability_of_paraphrase', ascending=False)

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
888,translate English to Malayalam,അടുത്തിടെ ആരംഭിച്ച അന്ത്യോദയ എക്‌സ്പ്രസിനെ പര...,അന്ത്യയോദയ എക്സ്പ്രൈസ് ന്റെ അടുത്ത പ്രക്ഷേപണത്...,0.499496,0.940987
5,translate English to Hindi,अपने संदेश में उन्होंने कहा है : – ‘‘विश्वे रे...,विश्व रेडियो दिवस पर बधाई।,0.498829,0.788068
481,translate English to Malayalam,യാദ്-ഇ-ജാലിന്‍ മ്യുസിയം 1919 ഏപ്രില്‍ 13ന് സംഭ...,"യാദ-ഇ-ജില്ലയന് മ്യൂസിയം, 1919 ஏப்ரல் 13ന് നടന്...",0.498121,0.853606
1062,translate English to Tamil,தென்னாப்பிரிக்காவின் நிற வெறியை எதிர்ப்பதில் இ...,தென்னாப்பிரிக்காவில் இனவாதத்தை எதிர்ப்பதில் நா...,0.498036,0.912589
1215,translate English to Tamil,"இந்தத் திட்டத்தால் தலித், மலைவாழ் மற்றும் பிற்...","இந்தத் திட்டத்தில் பல தாலித், பழங்குடி மற்றும்...",0.497654,0.962823
...,...,...,...,...,...
1171,translate English to Tamil,ரஷ்யாவின் தீவிரவாத்திற்கு எதிரான பார்வையே எங்க...,G20 க்குள் நமது ஒத்துழைப்பை மேம்படுத்தவும் நாங...,0.010659,0.887412
678,translate English to Malayalam,"റോഡുകള്‍, റെയില്‍വേ, ഒപ്റ്റിക്കല്‍ ഫൈബര്‍ ശൃംഖ...",He also mentioned the increase in pace in vari...,0.010443,0.539956
137,translate English to Malayalam,സാമ്പത്തിക പരിവര്‍ത്തനത്തിനും വികസനത്തിനും ഇന്...,"ഇന്ത്യയുടെ പുരോഗതിയിൽ നിന്നും, സമ്പന്നതയിൽ നിന...",0.009592,0.431183
55,translate English to Hindi,इसमें Ease of doing business के साथ ही Ease of...,व्यवसाय करने में आसानता और जीवन में आसानता इस ...,0.009105,0.711414


In [51]:
# Get the target sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'target_sentence'].iloc[0]

'தெற்கு குஜராத்தில் உள்ள வால்சாத், நவ்சாரி, தபி, சூரத், டாங்ஸ் ஆகிய ஐந்து மாவட்டங்களைச் சேர்ந்த பயனாளிகள் வால்சாத்தில் கூடுகின்றனர்.'

In [52]:
# Get the translation sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'bart_translation'].iloc[0]

'தெற்கு குஜராத்தின் ஐந்து மாவட்டங்களான வல்சாத், நவ்சாரி, தாபி, சூரத், டாங்ஸ் ஆகியவற்றில் பயனாளிகள் வல்சாட்டில் கூடியிருப்பார்கள்.'

In [53]:
# The English translation of the target sentence with the maximum cosine similarity score - using Google translate to translate to English
english_np_target = 'Beneficiaries from the five districts of Walsad, Navsari, Tabi, Surat and Tangs in South Gujarat gather in Walsad.'
english_np_translation = 'Beneficiaries will gather in Valsad from five districts of South Gujarat namely Valsad, Navsari, Tabi, Surat and Tangs.'
# The sentences have the same meaning, but the order is reversed - this represents an error of our model
print(english_np_target)
print(english_np_translation)

Beneficiaries from the five districts of Walsad, Navsari, Tabi, Surat and Tangs in South Gujarat gather in Walsad.
Beneficiaries will gather in Valsad from five districts of South Gujarat namely Valsad, Navsari, Tabi, Surat and Tangs.


In [54]:
# Let's look at the sentence pair with the lowest cosine similarity in the paraphrase dataset
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'target_sentence'].iloc[0])
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'bart_translation'].iloc[0])

आप सबका धन्यवाद!
बहुत धन्यवाद।


In [55]:
# The sentence meanings are similar - this is a correct labeling by our model
back_trans_target = 'Thank you all!'
back_trans_translation = 'Thanks a lot'

In [56]:
# Write the bart performance file to a csv 
bart_performance.to_csv(D+"/predicted_text/siamese_evaluations_of_translations/bart_siamese_paraphrase_eval_layernorm.csv")